In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 11.1 MB/s 
     |████████████████████████████████| 62 kB 2.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [16]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(how_many_future_candles, how_many_past_candles, each_row_past):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 1
  for file in files:
     print(f"File Number {index}:", end=" ")
     each_file_proc(file, now, how_many_future_candles,
                    how_many_past_candles, each_row_past)
     index = index + 1
  print(" ")
  return now


def each_file_proc(file, now, how_many_future_candles, how_many_past_candles, each_row_past):

    address = f"/content/data/{file}"
    data = pd.read_csv(address)
    data = data.dropna()
    dfs = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-which_future_or_past):

        rows = data[i-each_row_past:i, :4]

        past_candles = []
        for z in range(1, how_many_past_candles+1):
          past_candles.append(data[i-z][3])
        past_candles = sum(past_candles)/len(past_candles)
        next_candles = []
        for z in range(0, how_many_future_candles):
          next_candles.append(data[i+z][3])
        next_candles = sum(next_candles)/len(next_candles)
        if next_candles > past_candles:
          suggs.append(1)
        else:
          suggs.append(0)

        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close"])
        rows = None
        past_candles = None
        next_candles = None
        dfs.append(x)
        x = None
    data = None

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        rrng = np.array(df).shape[0]
        for i in range(rrng):
            date_list.append(datetime.fromordinal(
                datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        fig1, e = mpl.plot(df, type="candle", style="yahoo", axisoff=True,
                           scale_padding=0.5, returnfig=True, savefig=address, tight_layout=True)
        fig1.clear()
        date_list = None
        fig1 = None
        e = None
        right_now = None
        df = None
        index = index + 1
        if index % 10 == 0:
            print(f"{index}/{all_n}", end=" ")
        if index % 190 == 0:
          print(" ")
    print("")
    dfs = None
    suggs = None

def start(how_many_future_candles, how_many_past_candles, each_row_past):
    folder_name = extract_data(
        how_many_future_candles, how_many_past_candles, each_row_past)
    return folder_name


In [31]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd","ada-usd","bnb-usd","ltc-usd","xrp-usd"]
download_data(symbols,"max","1wk")

Symbols : 1500
Data Folder Removed
 


In [32]:

how_many_future_candles = 5
how_many_past_candles = 5
each_row_past = 50

global which_future_or_past
which_future_or_past = None
if how_many_future_candles > how_many_past_candles:
    which_future_or_past = how_many_future_candles
else:
    which_future_or_past = how_many_past_candles

folder_name = start(how_many_future_candles,how_many_past_candles,each_row_past)
len(os.listdir(f"/content/extracted/{folder_name}"))

Files In Data : 7
Processing File:
File Number 1: 10/191 20/191 30/191 40/191 50/191 60/191 70/191 80/191 90/191 100/191 110/191 120/191 130/191 140/191 150/191 160/191 170/191 180/191 190/191  

File Number 2: 10/191 20/191 30/191 40/191 50/191 60/191 70/191 80/191 90/191 100/191 110/191 120/191 130/191 140/191 150/191 160/191 170/191 180/191 190/191  

File Number 3: 10/355 20/355 30/355 40/355 50/355 60/355 70/355 80/355 90/355 100/355 110/355 120/355 130/355 140/355 150/355 160/355 170/355 180/355 190/355  
200/355 210/355 220/355 230/355 240/355 250/355 260/355 270/355 280/355 290/355 300/355 310/355 320/355 330/355 340/355 350/355 
File Number 4: 10/191 20/191 30/191 40/191 50/191 60/191 70/191 80/191 90/191 100/191 110/191 120/191 130/191 140/191 150/191 160/191 170/191 180/191 190/191  

File Number 5: 10/191 20/191 30/191 40/191 50/191 60/191 70/191 80/191 90/191 100/191 110/191 120/191 130/191 140/191 150/191 160/191 170/191 180/191 190/191  

File Number 6: 10/355 20/355 30/

1665

In [33]:
#folder_name = 132512
from PIL import Image
import cv2
label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name,0)
    scale_percent = 50 
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
np.array(data).shape

(1665, 277, 383)

In [34]:
def scaler(array):
  scaler = MinMaxScaler()
  scaler.fit(array)
  return scaler.transform(array)
data = np.array(data)
new_dd = []
for i in data:
  new_data = []
  for x in i:
    nx = scaler(np.array(x).reshape(-1,1))
    new_data.append(nx)
  new_dd.append(new_data)
np.array(new_dd).shape
data = np.array(new_dd).reshape(-1,data.shape[1],data.shape[2])
new_data = []

In [35]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
x_train , x_test , y_train , y_test = train_test_split(data,label,test_size=0.1)
data = None
label = None
print(f"********** {len(x_train)} {len(x_test)}")
nytrain = []
for i in y_train:
  if i == 1:
    nytrain.append([1,0])
  else:
    nytrain.append([0,1])
nytest = []
for i in y_test:
  if i == 1:
    nytest.append([1,0])
  else:
    nytest.append([0,1])

y_train = nytrain
y_test = nytest
x_train = np.array(x_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
y_train = np.array(y_train)

********** 1498 167


In [40]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense


model = Sequential()
model.add(Conv2D(32, (3, 3),activation="relu", input_shape=(277, 383,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(1024,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])



model.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 275, 381, 32)      320       
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 137, 190, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 135, 188, 32)      9248      
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 67, 94, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_37 (Conv2D)          (None, 65, 92, 32)        9248      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 32, 46, 32)      

In [ ]:
model.fit(x_train,y_train,batch_size=8,epochs=10,validation_data=(x_test,y_test))


Epoch 1/10
188/188 [==============================] - 184s 972ms/step - loss: 0.6999 - accuracy: 0.5561 - val_loss: 0.6540 - val_accuracy: 0.6108
Epoch 2/10
188/188 [==============================] - 183s 972ms/step - loss: 0.5865 - accuracy: 0.6836 - val_loss: 0.5056 - val_accuracy: 0.7246
Epoch 3/10
 75/188 [==========>...................] - ETA: 1:47 - loss: 0.4467 - accuracy: 0.7900